In [1]:
import telepot
from telepot.loop import MessageLoop
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton
from pprint import pprint
import sys
import time

bot = telepot.Bot('2067569208:AAHYKIRPZJ_eKtEFAtAmcFOWGlOMyRbGsXI')

In [2]:
def on_chat_message(msg):
    # Saving useful message info
    content_type, chat_type, chat_id = telepot.glance(msg)
    pprint(msg)
    
    # Check for content type
    if content_type == 'text':
        name = msg["from"]["first_name"]
        txt = str(msg['text'])
        if '/start' == txt:
            bot.sendMessage(chat_id, 'Hello {}! I\'m StackBot.'.format(name))

In [3]:
def on_callback_query(msg):
    query_id, from_id, query_data = telepot.glance(msg, flavor='callback_query')
    print("Callback Query: ", query_id, from_id, query_data)
    
    bot.answerCallbackQuery(query_id, text="YEAH")
    bot.sendMessage(query_id, "What are you looking for?")

In [ ]:
# Managing different inputs
bot.message_loop({'chat': on_chat_message,
                  'callback_query': on_callback_query})

# Keeping the program running
while 1:
    time.sleep(3)